In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
fake = Faker()

data = pd.read_csv('books_data.csv')
rating = pd.read_csv('Books_rating.csv')

In [2]:
print(len(data), len(rating))

212404 3000000


In [3]:
num_list = list(range(1000))
book_sold = [random.choice(num_list) for i in range(len(data))]
data['books_sold'] = book_sold

price_list = list(range(100, 1001))
prices = [random.choice(price_list) for i in range(len(data))]
data['price'] = prices

languages_list = ['Chinese', 'English', 'Spanish', 'Arabic', 'Hindi', 'Portuguese', 'Russian', 'Japanese', 'Urdu']
top_language = [random.choice(languages_list) for i in range(len(data))]
data['language'] = top_language

country_list = [fake.country() for i in range(len(data))]
country = pd.Series(country_list)
data['top_country'] = country

data['published_year'] = data['publishedDate'].apply(lambda x: str(x).split('-')[0])
data['published_year'] = data['published_year'].apply(lambda x: np.nan if '?' in x else x)
data['published_date'] = data['publishedDate'].apply(lambda x: np.nan if len(str(x).split('-')) != 3 else x)

data['ratingsCount'].replace(np.nan, 0, inplace=True)
data['ratings_count'] = data['ratingsCount'].apply(lambda x: int(x))

data.drop(['image', 'previewLink', 'infoLink', 'publishedDate', 'ratingsCount'], axis = 1, inplace=True)

data.reset_index(drop = True, inplace=True)


C:\Users\Satya Anudeep Kotaru\AppData\Local\Temp\ipykernel_17284\1891742958.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['ratingsCount'].replace(np.nan, 0, inplace=True)


In [4]:
rating['review/score'] = rating['review/score'].apply(lambda x: int(x))

user_email_list = [f"{row['User_id']}@user.com" for index, row in rating.iterrows()]
user_email = pd.Series(user_email_list)
rating['user_email'] = user_email

rating.drop(['Price', 'review/text'], axis = 1, inplace=True)
rating.rename(columns = {'review/helpfulness':'review_helpfulness', 'review/score':'review_score', 'review/time':'review_time', 'review/summary':'review_summary', 'review/text':'review_text', 'User_id':'user_id', 'profileName':'profile_name'}, inplace=True)

rating.reset_index(drop = True, inplace=True)

In [5]:
inner = pd.merge(data, rating, on='Title', how='inner')
inner.rename(str.lower, axis='columns', inplace=True)
print(inner.shape)
inner.head()

(3000000, 20)


,title,description,authors,publisher,categories,books_sold,price,language,top_country,published_year,published_date,ratings_count,id,user_id,profile_name,review_helpfulness,review_score,review_time,review_summary,user_email
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],NaN,['Comics & Graphic Novels'],410,122,Arabic,Palestinian Territory,1996,NaN,0,1882931173,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4,940636800,Nice collection of Julie Strain images,AVCGYZL8FQQTD@user.com
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,['Biography & Autobiography'],945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A30TK6U7DNS82R,Kevin Killian,10/10,5,1095724800,Really Enjoyed It,A30TK6U7DNS82R@user.com
2,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,['Biography & Autobiography'],945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A3UH4UZ4RSVO82,John Granger,10/11,5,1078790400,Essential for every personal and Public Library,A3UH4UZ4RSVO82@user.com
3,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,['Biography & Autobiography'],945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4,1090713600,Phlip Nel gives silly Seuss a serious treatment,A2MVUWT453QH61@user.com
4,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,['Biography & Autobiography'],945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4,1107993600,Good academic overview,A22X4XUPKF66MR@user.com


In [6]:
inner.drop(['description'], axis=1, inplace=True)

inner.replace('"', '', regex=True, inplace=True)
inner.replace("'", '', regex=True, inplace=True)
inner.replace(',', '', regex=True, inplace=True)

inner.dropna(inplace=True)


In [11]:
inner = inner[(inner != '').all(axis=1)]
inner.reset_index(inplace=True)
inner = inner.drop(['index'], axis=1)


In [15]:
names_list = [fake.unique.name() for i in range(len(inner['title'].unique()))]
names = pd.Series(names_list)
books = inner['title'].unique()

author_book = {key: value for key, value in zip(books, names)}
inner.drop(['authors'], axis=1, inplace=True)
inner['author'] = inner['title'].apply(lambda x: author_book[x])
inner['author_email'] = inner['author'].apply(lambda x: '_'.join(x.split(" ")) + '@gmail.com')

In [16]:
categories = inner['categories'].unique()

unique_categories = list()
for i in categories:
    j = i.replace('[', '').replace(']', '').lower()
    j = j.split('&')
    for k in j:
        unique_categories.append(k)

unique_categories = set(unique_categories)
unique_categories = list(unique_categories)
categories = [random.choice(unique_categories) for i in range(len(inner['title'].unique()))]
categories = pd.Series(categories)
books = inner['title'].unique()

book_category = {key: value for key, value in zip(books, categories)}
inner.drop(['categories'], axis=1, inplace=True)
inner['category'] = inner['title'].apply(lambda x: book_category[x])
inner['published_year'] = inner['published_year'].astype(int)

In [17]:
user_profiles = dict()
remove_profiles = dict()
def check_profiles(user_id, profile_name):
    if user_id in user_profiles.keys():
        if profile_name not in user_profiles[user_id]:
            user_profiles[user_id].append(profile_name)
            remove_profiles[user_id].append(profile_name)
    else:
        user_profiles[user_id] = [profile_name]
        remove_profiles[user_id] = list()

user_ids = inner.apply(lambda x: x['user_id'] if check_profiles(x['user_id'], x['profile_name']) else None, axis=1)

filtered_profiles = {key: value for key, value in remove_profiles.items() if value}

condition = inner.apply(lambda row: row['profile_name'] in filtered_profiles.get(row['user_id'], []), axis=1)

dataset = inner[~condition]

# indexes = inner.apply(lambda x: x.index if (x['user_id'] in filtered_profiles.keys()) and (x['profile_name'] in filtered_profiles[x['user_id']]) else 'remove', axis=1)

# profiles = dict()
# keys = list(user_profiles.keys())

# for i in range(len(keys)):
#     if len(user_profiles[keys[i]]) > 1:
#         profiles[keys[i]] = user_profiles[keys[i]][1:]

In [18]:
len(dataset)

1141524

In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1141524 entries, 0 to 1141852
Data columns (total 20 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   title               1141524 non-null  object
 1   publisher           1141524 non-null  object
 2   books_sold          1141524 non-null  int64 
 3   price               1141524 non-null  int64 
 4   language            1141524 non-null  object
 5   top_country         1141524 non-null  object
 6   published_year      1141524 non-null  int32 
 7   published_date      1141524 non-null  object
 8   ratings_count       1141524 non-null  int64 
 9   id                  1141524 non-null  object
 10  user_id             1141524 non-null  object
 11  profile_name        1141524 non-null  object
 12  review_helpfulness  1141524 non-null  object
 13  review_score        1141524 non-null  int64 
 14  review_time         1141524 non-null  int64 
 15  review_summary      1141524 non-null 

In [20]:
dataset.head()

,title,publisher,books_sold,price,language,top_country,published_year,published_date,ratings_count,id,user_id,profile_name,review_helpfulness,review_score,review_time,review_summary,user_email,author,author_email,category
0,Dr. Seuss: American Icon,A&C Black,945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A30TK6U7DNS82R,Kevin Killian,10/10,5,1095724800,Really Enjoyed It,A30TK6U7DNS82R@user.com,Jason Carpenter,Jason_Carpenter@gmail.com,year 2000 date conversion (computer systems)
1,Dr. Seuss: American Icon,A&C Black,945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A3UH4UZ4RSVO82,John Granger,10/11,5,1078790400,Essential for every personal and Public Library,A3UH4UZ4RSVO82@user.com,Jason Carpenter,Jason_Carpenter@gmail.com,year 2000 date conversion (computer systems)
2,Dr. Seuss: American Icon,A&C Black,945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A2MVUWT453QH61,Roy E. Perry amateur philosopher,7/7,4,1090713600,Phlip Nel gives silly Seuss a serious treatment,A2MVUWT453QH61@user.com,Jason Carpenter,Jason_Carpenter@gmail.com,year 2000 date conversion (computer systems)
3,Dr. Seuss: American Icon,A&C Black,945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A22X4XUPKF66MR,D. H. Richards ninthwavestore,3/3,4,1107993600,Good academic overview,A22X4XUPKF66MR@user.com,Jason Carpenter,Jason_Carpenter@gmail.com,year 2000 date conversion (computer systems)
4,Dr. Seuss: American Icon,A&C Black,945,179,Japanese,Oman,2005,2005-01-01,0,0826414346,A2F6NONFUDB6UK,Malvin,2/2,4,1127174400,One of Americas greatest creative talents,A2F6NONFUDB6UK@user.com,Jason Carpenter,Jason_Carpenter@gmail.com,year 2000 date conversion (computer systems)


In [21]:
dataset.to_csv('AmazonBooks.csv', index = False)

In [8]:
df[df['user_id'] == 'A67HJ7CPI0S21']['review_summary'] == ''

Series([], Name: review_summary, dtype: bool)